# Structured Outputs: From Text to Tabular Data

### Import Libraries

In [2]:
from ollama import chat
from pydantic import BaseModel
import pandas as pd
from gnews import GNews

### Pull News Headline Data

In [ ]:

# Fetch news articles
google_news = GNews()
news = google_news.get_news("NVDA")

# Extract top 6 news titles
news_titles = [article['title'] for article in news[:6]]
news_titles

### LLM Model and Structured Outputs

In [ ]:
# Define BaseModel for news analysis
class NewsAnalysis(BaseModel):
    sentiment: str  
    future_looking: bool  

# Initialize an empty list to store results
results = []

# Loop through the news titles and analyze each
for title in news_titles:
    response = chat(
        messages=[
            {
                'role': 'user',
                'content': f"""Analyze the following title for sentiment (positive, negative, or neutral) 
                               and whether it provides future-looking financial insight, predictions, or 
                               guidance on whether to buy/hold/sell the stock (True or False): {title}
                """,
            }
        ],
        model='llama3.2',
        format=NewsAnalysis.model_json_schema(),
    )

    # Parse the response into the NewsAnalysis model
    sentiment_analysis = NewsAnalysis.model_validate_json(response['message']['content'])

    # Append the results to the list
    results.append({
        'title': title,
        'sentiment': sentiment_analysis.sentiment,
        'future_looking': sentiment_analysis.future_looking
    })

# Convert the results to a DataFrame
df = pd.DataFrame(results)
df
